## Graph

### MAE: random_masking

In [148]:
x = torch.rand((4, 5, 2))
x

tensor([[[0.4242, 0.2900],
         [0.3482, 0.5577],
         [0.2255, 0.1098],
         [0.8989, 0.5212],
         [0.7968, 0.0127]],

        [[0.3815, 0.1038],
         [0.2913, 0.9885],
         [0.6024, 0.2209],
         [0.1044, 0.9607],
         [0.6929, 0.2405]],

        [[0.1717, 0.6199],
         [0.4443, 0.0392],
         [0.0725, 0.5095],
         [0.2704, 0.8845],
         [0.1892, 0.1595]],

        [[0.6444, 0.4932],
         [0.7901, 0.1293],
         [0.1907, 0.7914],
         [0.7692, 0.4905],
         [0.1445, 0.6353]]])

In [62]:
noise = torch.rand(4, 5)
noise

tensor([[0.0835, 0.5561, 0.9291, 0.7978, 0.9702],
        [0.5953, 0.2089, 0.5204, 0.3862, 0.4703],
        [0.1807, 0.2943, 0.8598, 0.4118, 0.7235],
        [0.1801, 0.3224, 0.7655, 0.5698, 0.9442]])

In [63]:
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_shuffle

tensor([[0, 1, 3, 2, 4],
        [1, 3, 4, 2, 0],
        [0, 1, 3, 4, 2],
        [0, 1, 3, 2, 4]])

In [64]:
ids_restore = torch.argsort(ids_shuffle, dim=1)
ids_restore

tensor([[0, 1, 3, 2, 4],
        [4, 0, 3, 1, 2],
        [0, 1, 4, 2, 3],
        [0, 1, 3, 2, 4]])

In [65]:
ids_keep = ids_shuffle[:, :3]
ids_keep

tensor([[0, 1, 3],
        [1, 3, 4],
        [0, 1, 3],
        [0, 1, 3]])

In [66]:
x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, 2))
x_masked

tensor([[[0.0841, 0.7777],
         [0.2600, 0.6878],
         [0.5623, 0.3064]],

        [[0.5746, 0.9682],
         [0.3447, 0.8661],
         [0.1676, 0.4896]],

        [[0.4387, 0.9735],
         [0.8970, 0.7504],
         [0.7708, 0.4949]],

        [[0.0260, 0.9256],
         [0.9354, 0.9779],
         [0.6392, 0.9462]]])

In [67]:
mask = torch.ones([4, 5], device=x.device)
mask[:, :3] = 0
# unshuffle to get the binary mask
mask = torch.gather(mask, dim=1, index=ids_restore)
mask

tensor([[0., 0., 1., 0., 1.],
        [1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1.],
        [0., 0., 1., 0., 1.]])

### Debug:

In [1]:
import moco as moco

print(moco.__dict__['moco_graph_encoder'])

<function moco_graph_encoder at 0x7f81808a4790>


In [13]:
h_pl = torch.rand((4, 2))
c = torch.rand((4, 2))
print(h_pl)
print(c)

tensor([[0.0808, 0.9130],
        [0.8778, 0.8021],
        [0.0944, 0.6027],
        [0.2141, 0.1238]])
tensor([[0.8555, 0.2620],
        [0.9656, 0.1462],
        [0.1758, 0.5150],
        [0.4018, 0.3649]])


In [14]:
import torch.nn as nn

n_h = 2
f_k = nn.Bilinear(n_h, n_h, 1)

scs = []
scs.append(f_k(h_pl, c))
scs

[tensor([[ 0.3055],
         [ 0.4926],
         [-0.1603],
         [-0.1570]], grad_fn=<AddBackward0>)]

In [15]:
c_mi = c
negsamp_round = 1
for _ in range(negsamp_round):
    c_mi = torch.cat((c_mi[-2:-1,:], c_mi[:-1,:]),0)
    print(c_mi)
    scs.append(f_k(h_pl, c_mi))

tensor([[0.1758, 0.5150],
        [0.8555, 0.2620],
        [0.9656, 0.1462],
        [0.1758, 0.5150]])


In [17]:
print(scs)

[tensor([[ 0.3055],
        [ 0.4926],
        [-0.1603],
        [-0.1570]], grad_fn=<AddBackward0>), tensor([[-0.1441],
        [ 0.3266],
        [ 0.2137],
        [-0.2137]], grad_fn=<AddBackward0>)]


In [16]:
logits = torch.cat(tuple(scs))
logits

tensor([[ 0.3055],
        [ 0.4926],
        [-0.1603],
        [-0.1570],
        [-0.1441],
        [ 0.3266],
        [ 0.2137],
        [-0.2137]], grad_fn=<CatBackward0>)

In [18]:
x = torch.rand((4, 5, 2))
x

tensor([[[0.2789, 0.2368],
         [0.8589, 0.8995],
         [0.1355, 0.6173],
         [0.6932, 0.8482],
         [0.6499, 0.5748]],

        [[0.4149, 0.4989],
         [0.7000, 0.2095],
         [0.5814, 0.5955],
         [0.2664, 0.6382],
         [0.3337, 0.7935]],

        [[0.2979, 0.6018],
         [0.3499, 0.3546],
         [0.2195, 0.0341],
         [0.2673, 0.9819],
         [0.3262, 0.5714]],

        [[0.9486, 0.0165],
         [0.0677, 0.1777],
         [0.0727, 0.1079],
         [0.3486, 0.1212],
         [0.0447, 0.4711]]])

In [19]:
x[:, 0, :]

tensor([[0.2789, 0.2368],
        [0.4149, 0.4989],
        [0.2979, 0.6018],
        [0.9486, 0.0165]])

In [57]:
from dgl.nn.pytorch.glob import AvgPooling

avg = AvgPooling()

out = avg(temp, temp.ndata["features"])
print(out.shape)

torch.Size([1, 1433])


In [58]:
out = out.to(torch.float32)

In [59]:
drop = nn.Dropout(0.2)
linear = nn.Linear(1433, 32)
out = drop(linear(out))
print(out.shape)

torch.Size([1, 32])


In [60]:
score_over_layer = 0
score_over_layer += out

In [62]:
score_over_layer.shape

torch.Size([1, 32])

In [91]:
import torch

# Sinkhorn-Knopp
def sinkhorn(scores, eps=0.05, niters=3):
    Q = torch.exp(scores / eps).T
    Q /= sum(Q)
    K, B = Q.shape
    u, r, c = torch.zeros(K), torch.ones(K) / K, torch.ones(B) / B
    for _ in range(niters):
        u = torch.sum(Q, dim=1)
        Q *= (r / u).unsqueeze(1)
        Q *= (c / torch.sum(Q, dim=0)).unsqueeze(0)
    return (Q / torch.sum(Q, dim=0, keepdim=True)).T

z = torch.randn((4, 2))
print(z)

tensor([[-0.4061, -0.9381],
        [-0.6663,  0.9505],
        [ 0.6383,  1.0689],
        [ 1.8522, -0.2765]])


In [93]:
x = sinkhorn(z)
x

tensor([[9.9998e-01, 2.3970e-05],
        [9.0603e-15, 1.0000e+00],
        [1.8182e-04, 9.9982e-01],
        [1.0000e+00, 3.2391e-19]])

In [150]:
a = torch.rand((1, 4))
print(a)
b = F.softmax(a, dim=1)
print(b)

tensor([[0.8667, 0.7765, 0.2961, 0.1600]])
tensor([[0.3365, 0.3074, 0.1902, 0.1660]])


In [151]:
T = 0.8
numerator = np.exp(a) / T
print(numerator)

tensor([[2.9739, 2.7173, 1.6808, 1.4670]])


### Mahalanobis

In [268]:
# load networks
from resnet import ResNet34
model = ResNet34(num_c=10)

# set information about feature extaction
model.eval()
temp_x = torch.rand(32, 3, 32, 32)
temp_x = temp_x.detach()
temp_list = model.feature_list(temp_x)[1] # (B, C, H, W)
num_output = len(temp_list)
print("num of layers: ", num_output)
for i in range(len(temp_list)):
    print("size of each layer: ", temp_list[i].shape)

feature_list = np.empty(num_output) # [0] * num_layers
count = 0
for out in temp_list:
    feature_list[count] = out.size(1) # C
    count += 1
print("feature_list:", feature_list)

num of layers:  5
size of each layer:  torch.Size([32, 64, 32, 32])
size of each layer:  torch.Size([32, 64, 32, 32])
size of each layer:  torch.Size([32, 128, 16, 16])
size of each layer:  torch.Size([32, 256, 8, 8])
size of each layer:  torch.Size([32, 512, 4, 4])
feature_list: [ 64.  64. 128. 256. 512.]


In [269]:
out_features = temp_list
for i in range(num_output):
    out_features[i] = out_features[i].view(out_features[i].size(0), out_features[i].size(1), -1) # (B, C, H*W)
    out_features[i] = torch.mean(out_features[i].data, 2) # (B, C)
    print(out_features[i].data.shape)

torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 128])
torch.Size([32, 256])
torch.Size([32, 512])


In [271]:
num_classes = 10
num_sample_per_class = np.empty(num_classes) # [0] * 10
num_sample_per_class.fill(0)
list_features = []
for i in range(num_output):
    temp_list = [] # [0] * 10
    for j in range(num_classes):
        temp_list.append(0)
    list_features.append(temp_list) # [[0] * 10] * num_layers

target = torch.randint(0, 10, (32, ))
data = temp_x

# construct the sample matrix
for i in range(data.size(0)): # [0, B-1]
    label = target[i]
    if num_sample_per_class[label] == 0:
        out_count = 0
        for out in out_features: # out: (B, num_feature)
            list_features[out_count][label] = out[i].view(1, -1) # (1, num_feature)
            out_count += 1
    else:
        out_count = 0
        for out in out_features:
            list_features[out_count][label] \
            = torch.cat((list_features[out_count][label], out[i].view(1, -1)), 0)
            out_count += 1                
    num_sample_per_class[label] += 1

In [275]:
# (num_layers, num_class, N_c, C)
print(len(list_features))
print(len(list_features[0]))
tmp = 0
for i in range(num_classes):
    tmp += len(list_features[0][i])
print(tmp)
print(len(list_features[0][0][0]))

5
10
32
64


In [286]:
sample_class_mean = [] # [(num_classes, C)] * num_layers
out_count = 0
for num_feature in feature_list: # num_layers
    temp_list = torch.Tensor(num_classes, int(num_feature)) # (num_classes, C)
    for j in range(num_classes):
        temp_list[j] = torch.mean(list_features[out_count][j], 0) # (N_c, C) -> (1, C)
    sample_class_mean.append(temp_list)
    out_count += 1
print(len(sample_class_mean))
print(sample_class_mean[0].shape)

5
torch.Size([10, 64])


In [293]:
import sklearn.covariance

group_lasso = sklearn.covariance.EmpiricalCovariance(assume_centered=False)

precision = []
for k in range(num_output):
    X = 0
    for i in range(num_classes):
        if i == 0:
            X = list_features[k][i] - sample_class_mean[k][i] # (N_c, C)
        else:
            X = torch.cat((X, list_features[k][i] - sample_class_mean[k][i]), 0) # (N, C)
    # find inverse            
    group_lasso.fit(X.numpy())
    temp_precision = group_lasso.precision_
    temp_precision = torch.from_numpy(temp_precision).float() # (C, N)
    precision.append(temp_precision) # [(C, N)] * num_layers
print(len(precision))
print(precision[0].shape) # [(C, C)] * num_layers

5
torch.Size([64, 64])


In [345]:
x = torch.randn((3, 2)) # (B, C)
sigma = torch.randn((2, 2)) # (C, C)
print(x)
print(sigma)

tensor([[ 0.4317,  0.1070],
        [-0.3850, -0.6187],
        [ 0.4838,  0.0031]])
tensor([[-2.2033,  2.2806],
        [ 0.0719,  1.6152]])


In [353]:
y = torch.mm(torch.mm(x, sigma), x.t()).diag()
y = y.view(-1, 1)
print(y.shape)

torch.Size([3, 1])


In [358]:
t = torch.randn((3, 1))
t = torch.cat((y, t), 1)
print(t)
print(t.shape)

tensor([[-0.2834,  0.0621],
        [ 0.8519, -0.5122],
        [-0.5122,  0.8572]])
torch.Size([3, 2])


In [361]:
t.max(1)

torch.return_types.max(
values=tensor([0.0621, 0.8519, 0.8572]),
indices=tensor([1, 0, 1]))

In [374]:
x = torch.randn((4, 2))
y = torch.randn((4, 2))
z = torch.sum(x * y, dim=1)
print(z.shape)
z

torch.Size([4])


tensor([ 0.1002, -1.0799,  0.3467,  0.8103])

In [376]:
for i in range(4):
    print(sum(x[i]*y[i]))    

tensor(0.1002)
tensor(-1.0799)
tensor(0.3467)
tensor(0.8103)


In [12]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score

# 生成一些示例数据
np.random.seed(42)
X = np.random.rand(100, 2)

# 计算协方差矩阵的逆
cov_matrix_inv = np.linalg.inv(np.cov(X, rowvar=False))

# 中心化数据集
mean = np.mean(X, axis=0)
X_centered = X - mean

# 变换数据集
X_transformed = np.dot(X_centered, cov_matrix_inv)

# 使用 scikit-learn 的 KMeans 基于欧氏距离
kmeans_euclidean = KMeans(n_clusters=2, init='k-means++', n_init='auto', algorithm='lloyd', random_state=42)
kmeans_euclidean.fit(X)

# 使用 scikit-learn 的 KMeans 基于马氏距离
kmeans_mahalanobis = KMeans(n_clusters=2, init='k-means++', n_init='auto', algorithm='lloyd', random_state=42)
kmeans_mahalanobis.fit(X_transformed)

# 获取聚类结果
labels_euclidean = kmeans_euclidean.labels_
labels_mahalanobis = kmeans_mahalanobis.labels_

# 打印聚类结果
print("基于欧氏距离的 K-means 聚类结果：", labels_euclidean)
print("基于马氏距离的 K-means 聚类结果：", labels_mahalanobis)

# 计算基于马氏距离的 K-means 聚类的轮廓系数
silhouette_score_ou = silhouette_score(X, labels_mahalanobis)
silhouette_score_mahalanobis = silhouette_score(X_transformed, labels_mahalanobis)

# 打印基于马氏距离的 K-means 聚类的轮廓系数
print("基于马氏距离的 K-means 聚类的轮廓系数：", silhouette_score_ou)
print("基于马氏距离的 K-means 聚类的轮廓系数：", silhouette_score_mahalanobis)


基于欧氏距离的 K-means 聚类结果： [0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 1 0
 0 1 1 0 1 1 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0
 1 1 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0]
基于马氏距离的 K-means 聚类结果： [0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 0 1 0
 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0
 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0]
基于马氏距离的 K-means 聚类的轮廓系数： 0.3397246387107845
基于马氏距离的 K-means 聚类的轮廓系数： 0.3736980583593435


### 调试

In [6]:
import math
import dgl
import time
import torch
import numpy as np
import random
import warnings
# from functools import partial
from utils import *
import clustering
from dataset import GraphDataset
from model import dmc
from sklearn.metrics import roc_auc_score

In [4]:
import torch
import numpy as np
from utils import *
from dataset import GraphDataset

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
adj, features, labels = load("cora")
# adj = adj.todense()
features = preprocess_features(features)
features = torch.FloatTensor(features)
print("feature_dim: ", features.shape)
# adj = torch.FloatTensor(adj)
labels = torch.FloatTensor(labels)

train_dataset = GraphDataset(adj=adj, features=features, aug=None)
train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=256,
        collate_fn=labeled_batcher(),
        shuffle=True,
        pin_memory=True,
        sampler=None,
        drop_last=True
    )

feature_dim:  torch.Size([2708, 1433])


In [5]:
for g, y in train_loader:
    print(y.shape)
    print(g)
    break

13
torch.Size([13, 1433])
13
13
torch.Size([13, 1433])
13
8
torch.Size([8, 1433])
8
11
torch.Size([11, 1433])
11
14
torch.Size([14, 1433])
14
10
torch.Size([10, 1433])
10
15
torch.Size([15, 1433])
15
21
torch.Size([21, 1433])
21
9
torch.Size([9, 1433])
9
10
torch.Size([10, 1433])
10
11
torch.Size([11, 1433])
11
13
torch.Size([13, 1433])
13
5
torch.Size([5, 1433])
5
21
torch.Size([21, 1433])
21
3
torch.Size([3, 1433])
3
9
torch.Size([9, 1433])
9
13
torch.Size([13, 1433])
13
19
torch.Size([19, 1433])
19
12
torch.Size([12, 1433])
12
7
torch.Size([7, 1433])
7
4
torch.Size([4, 1433])
4
16
torch.Size([16, 1433])
16
10
torch.Size([10, 1433])
10
7
torch.Size([7, 1433])
7
4
torch.Size([4, 1433])
4
3
torch.Size([3, 1433])
3
9
torch.Size([9, 1433])
9
5
torch.Size([5, 1433])
5
9
torch.Size([9, 1433])
9
7
torch.Size([7, 1433])
7
13
torch.Size([13, 1433])
13
10
torch.Size([10, 1433])
10
10
torch.Size([10, 1433])
10
10
torch.Size([10, 1433])
10
21
torch.Size([21, 1433])
21
8
torch.Size([8, 1433])
8
5

In [6]:
step_dist=[1.0, 0.0, 0.0]
step = np.random.choice(len(step_dist), 1, p=step_dist)[0]
step

0

In [4]:
g = dgl.DGLGraph()
g.add_nodes(3)
g.add_edges([0, 0, 1, 2], [1, 2, 2, 1])
g.ndata["pos_directed"] = torch.rand(3, 16)
g.ndata["pos_undirected"] = torch.rand(3, 16)
g.ndata["seed"] = torch.zeros(3, dtype=torch.long)
g.ndata["nfreq"] = torch.ones(3, dtype=torch.long)
g.edata["efreq"] = torch.ones(4, dtype=torch.long)
g = dgl.batch([g, g, g])

In [6]:
adj_matrix = g.adjacency_matrix_scipy(transpose=False, return_edge_ids=False).astype(float)
adj_matrix

<9x9 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [20]:
node_idx = np.array([0, 1, 2, 3])
other_node_idx = node_idx

In [8]:
traces = dgl.contrib.sampling.random_walk_with_restart(
        g,
        seeds=[0],
        restart_prob=0.8,
        max_nodes_per_seed=10,
    )

In [12]:
traces[0]

(tensor([2]),
 tensor([2, 1]),
 tensor([2]),
 tensor([1]),
 tensor([2]),
 tensor([1, 2, 1]),
 tensor([2]))